In [1]:
import devito
from matplotlib import pyplot as plt
import numpy as np
%matplotlib inline

In [2]:
Lx = 100.0   # width of the mantle in the x direction []
Ly = 100.0   # thickness of the mantle in the y direction []
nx = 401     # number of points in the x direction
ny = 401     # number of points in the y direction
dx = Lx / (nx - 1)  # grid spacing in the x direction []
dy = Ly / (ny - 1)  # grid spacing in the y direction []
g = 9.81  # gravity acceleration [m/s^2]
alpha = 0.025 # friction coefficient for natural channels in good condition
# Maximum wave propagation time [s]
Tmax = 6.
dt = 1/4500.
nt = (int)(Tmax/dt)

In [3]:
from devito import Grid, TimeFunction, Function

grid = Grid(shape=(ny, nx), extent=(Ly, Lx))
eta = TimeFunction(name='eta', grid=grid)
M = TimeFunction(name='M', grid=grid)
N = TimeFunction(name='N', grid=grid)
h = Function(name='h', grid=grid)

In [4]:
D = eta + h
frictionTerm = g * alpha**2/D**(7/3)*(M**2+N**2)**(-1/2)

In [46]:
import numpy
# Define the locations along a gridline.
x = numpy.linspace(0.0, Lx, num=nx)
y = numpy.linspace(0.0, Ly, num=ny)

# Define initial eta, M, N
X, Y = numpy.meshgrid(x,y) # coordinates X,Y required to define eta, h, M, N

# Define constant ocean depth profile h = 50 m
h.data[:] = 50 * numpy.ones_like(X)

# Define initial eta Gaussian distribution [m]
eta0 = 0.5 * numpy.exp(-((X-50)**2/10)-((Y-50)**2/10))

# Define initial M and N
M0 = 100. * eta0
N0 = 0. * M0

In [5]:
from devito import Eq, solve
pde_eta = eta.dt + M.dx + N.dy
eq_eta = Eq(eta.forward, solve(pde_eta, eta.forward))
pde_M = M.dt + (M**2/D).dx + (M*N/D).dy + g*D*eta.dx + frictionTerm*M
eq_M = Eq(M.forward, solve(pde_M, M.forward))
pde_N = N.dt + (M*N/D).dx + (N**2/D).dy + g*D*eta.dy + frictionTerm*N
eq_N = Eq(N.forward, solve(pde_N, N.forward))

In [62]:
from devito import Operator
eta.data[0] = eta0
M.data[0]   = M0
N.data[0]   = N0

op = Operator([eq_eta] + [eq_M] + [eq_N])

In [ ]:
op.apply(time=nt, dt=dt)